In [1]:
!python ../setup.py build 
!python ../setup.py build_ext --inplace

First: /Users/hyunseongkim/Documents/GitHub/Org_OptTrot/OptTrot
/Users/hyunseongkim/Documents/GitHub/Org_OptTrot/OptTrot/opttrot
running build
running build_py
file /Users/hyunseongkim/Documents/GitHub/Org_OptTrot/OptTrot/opttrot/opttrot.py (for module opttrot) not found
copying /Users/hyunseongkim/Documents/GitHub/Org_OptTrot/OptTrot/opttrot/graph_algs/dwave.py -> build/lib.macosx-11.1-arm64-cpython-312/graph_algs
copying /Users/hyunseongkim/Documents/GitHub/Org_OptTrot/OptTrot/opttrot/graph_algs/graph_opts.py -> build/lib.macosx-11.1-arm64-cpython-312/graph_algs
copying /Users/hyunseongkim/Documents/GitHub/Org_OptTrot/OptTrot/opttrot/graph_algs/__init__.py -> build/lib.macosx-11.1-arm64-cpython-312/graph_algs
file /Users/hyunseongkim/Documents/GitHub/Org_OptTrot/OptTrot/opttrot/opttrot.py (for module opttrot) not found
running build_ext
building 'c_utils' extension
clang -fno-strict-overflow -DNDEBUG -O2 -Wall -fPIC -O2 -isystem /Users/hyunseongkim/miniconda3/envs/opttrot/include -ar

In [1]:
import os, sys
sys.path.append("..")

## Vectorized commute

In [4]:
import opttrot
from opttrot.pauli import PauliElement, PauliPoly
from opttrot.pauli import get_commutes, get_commutes_sparse
import numpy as np

In [5]:
arr = np.array([2**64+1])
arr

array([18446744073709551617], dtype=object)

In [6]:
a = PauliElement(0, 1, 2, 1.)
b = PauliElement(2, 1, 2, 1.)
c = PauliElement(3, 0, 2, 1)
d = PauliElement(0,0,2,1)
a, b, c

(PauliElement(n=2, weight=1.000000+(0.000000)j, IZ),
 PauliElement(n=2, weight=1.000000+(0.000000)j, XZ),
 PauliElement(n=2, weight=1.000000+(0.000000)j, XX))

In [7]:
ppoly = PauliPoly.from_iterables((a,b,c, d))

In [8]:
ppoly.poly

array([PauliElement(n=2, weight=1.000000+(0.000000)j, II),
       PauliElement(n=2, weight=1.000000+(0.000000)j, XX),
       PauliElement(n=2, weight=1.000000+(0.000000)j, IZ),
       PauliElement(n=2, weight=1.000000+(0.000000)j, XZ)], dtype=object)

In [9]:
a.commute(a), a.commute(b), a.commute(c)

(True, True, False)

In [10]:
get_commutes(ppoly.poly, a)

array([ True, False,  True,  True])

In [11]:
from scipy.sparse import coo_matrix
import numpy as np

In [12]:
coo = coo_matrix(ppoly.coef_matrix)

In [13]:
%%timeit
a.commute(b)

213 ns ± 1.94 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [14]:
get_commutes_sparse(coo.row.astype(np.uint64), coo.col.astype(np.uint64), a)

array([ True, False,  True,  True])

In [15]:
nx_s = np.bitwise_xor(coo.row, coo.col)
nz_s = coo.col

In [16]:
nx, nz = a.sym_code

In [17]:
from opttrot.utils import np_bitwise_count

In [18]:
commute_arr = np.bitwise_xor(np.bitwise_and(nx_s, nz), np.bitwise_and(nz_s, nx))
if commute_arr.max() !=1:
    commute_arr = np_bitwise_count(commute_arr)

In [19]:
np.bitwise_xor(np.array([1,1,1]), 1)

array([0, 0, 0])

In [20]:
%%timeit
get_commutes(ppoly.poly, a)

954 ns ± 13.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [21]:
%%timeit
get_commutes_sparse(coo.row.astype(np.uint64), coo.col.astype(np.uint64), a)

1.67 μs ± 20.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [22]:
%%timeit
p = ppoly.poly
n = np.zeros(len(p))
for i in range(len(p)):
    n[i] = a.commute(p[i])

1.32 μs ± 23.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


# Optimization of Pauli Poly

There are two measure to estimate performances of `PauliPoly`.

- Naive `PauliPoly` implementation.
- Qiskit `SparsePauliOp` and `PauliList`.
- 
The PauliPoly in the origial OptTrot library was a naive implementation written by python.
In here, we will optimize the routine as soon as possible using scipy and numba routines.

In [23]:
import opttrot

In [24]:
from opttrot.pauli_c import get_paulilist_from_coefs

In [25]:
from opttrot.pauli_c import PauliElement
from opttrot.pauli_c import get_commutes

In [26]:
from opttrot.pauli import PauliElement
from opttrot.pauli import PauliPoly

import numpy as np
import matplotlib.pyplot as plt

In [27]:
from importlib import reload
#import paulipoly
#reload(paulipoly)
#from paulipoly import PauliPoly as NewPauliPoly
#import ten_con
#reload(ten_con)
#from ten_con import _mat_to_coef_mat, _coef_to_mat

In [28]:
from scipy.sparse import coo_matrix, csr_matrix

In [29]:
from qiskit.quantum_info import Pauli, PauliList, SparsePauliOp

In [30]:
from copy import deepcopy

In [31]:
qubits = 4
def get_test_hermit(qubits, tol= 0.5):
    ndim = int(2**qubits)
    A = np.random.rand(ndim, ndim)
    A[A<tol] = 0.
    H = A + 1j*A
    H = H.conj().T@H
    return H


In [32]:
A = np.sqrt(get_test_hermit(qubits, 0.96))
B = np.sqrt(get_test_hermit(qubits, 0.8))


In [33]:
test_ppoly = PauliPoly.from_matrix(A)

In [34]:
test_ppoly.terms

[((0.6841421933386542-1.0613673619882273e-18j), 'IIII'),
 ((0.1739606185042296-9.131559250095902e-19j), 'IIXI'),
 ((0.15798951542683504-4.413262847088973e-18j), 'IIIZ'),
 ((0.1739606185042296-9.131559250095902e-19j), 'IIXZ'),
 ((-0.2594954596351013-2.223976572674116e-18j), 'IIZI'),
 ((0.2666572182767179+1.12791891242663e-18j), 'IIZZ'),
 ((-0.1568075909944165+2.460764882443115e-19j), 'IZII'),
 ((-0.1739606185042296+9.131559250095902e-19j), 'IZXI'),
 ((-0.33289520614816404+7.153269704083757e-19j), 'IZIZ'),
 ((-0.1739606185042296+9.131559250095902e-19j), 'IZXZ'),
 ((0.08458976891377235-1.4739593040064815e-18j), 'IZZI'),
 ((0.2606773840675199-1.943209786170546e-18j), 'IZZZ'),
 ((-0.08768812241260611-1.1147544403641606e-18j), 'ZIII'),
 ((-0.1739606185042296+9.131559250095902e-19j), 'ZIXI'),
 ((0.08971508475602215+7.870602262344152e-19j), 'ZIIZ'),
 ((-0.1739606185042296+9.131559250095902e-19j), 'ZIXZ'),
 ((0.015470300331961964-1.1312837539800942e-19j), 'ZIZI'),
 ((-0.16193290683666628-2.0149

In [35]:
#plist = list(test_ppoly._terms.values())
coef_mat = test_ppoly.coef_matrix
mat = test_ppoly.matrix
coef_mat[coef_mat<0.05] = 0.
cmat = csr_matrix(coef_mat)
100*cmat.data.size/(4**qubits)

5.46875

In [36]:
from opttrot.pauli import PauliElement

In [37]:
a = PauliElement(3, 4, 5, 0.5)
b = PauliElement(1, 2, 5, 0.5)
c = PauliElement(2, 2, 5, 0.1)

In [38]:
parr = np.array([a,b, c])
print(parr)

[PauliElement(n=5, weight=0.500000+(0.000000)j, IIZXX)
 PauliElement(n=5, weight=0.500000+(0.000000)j, IIIZX)
 PauliElement(n=5, weight=0.100000+(0.000000)j, IIIYI)]


In [39]:
ppoly = PauliPoly.from_iterables(parr)

In [40]:
ppoly.poly

array([PauliElement(n=5, weight=0.100000+(0.000000)j, IIIYI),
       PauliElement(n=5, weight=0.500000+(0.000000)j, IIIZX),
       PauliElement(n=5, weight=0.500000+(0.000000)j, IIZXX)],
      dtype=object)

In [41]:
a.commute(b)

False

In [42]:
a+b

PauliElement(n=5, weight=0.500000+(0.000000)j, IIZXX)

In [43]:
c.commute(a), c.commute(b)

(False, False)

In [44]:
c.commute(ppoly)

False

## Coef matrix to PauliPoly

In [45]:
%%timeit
PauliPoly(coef_mat)

3.53 μs ± 87.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Hermit matrix to PauliPoly

In [46]:
%%timeit
PauliPoly.from_matrix(mat)

28.1 μs ± 326 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [47]:
%%timeit
SparsePauliOp.from_operator(mat)

10.8 μs ± 22.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Test

## PauliPoly to Hermit

In [48]:
# PauliPoly
pp001 = PauliPoly.from_matrix(A)
pp002 = PauliPoly.from_matrix(B)

# Qiskit
spa001 = SparsePauliOp.from_operator(A)
spa002 = SparsePauliOp.from_operator(B)

In [49]:
%%timeit
pp001.matrix

17.9 μs ± 328 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [50]:
%%timeit
spa001.to_matrix() 

22.6 μs ± 224 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Tset: Qubit 12 (Matrix construction)

- PauliPoly: 3.48 s ± 12.3 ms
- Qiskit: 33.2 s ± 1.98 s

In [51]:
%%timeit
pp002.matrix

19.1 μs ± 3.12 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [26]:
%%timeit
spa002.to_matrix()

96.8 µs ± 561 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## PauliElement generation

In [52]:
pp001.matrix

matrix([[ 1.40971565e+00-1.80529070e-17j,
          0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j],
        [ 0.00000000e+00+0.00000000e+00j,
          2.77555756e-17-7.70371978e-34j,
          0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.000000

In [53]:
ps = pp001.poly

In [54]:
ps

array([PauliElement(n=4, weight=0.684142+(-0.000000)j, IIII),
       PauliElement(n=4, weight=0.173961+(-0.000000)j, IIXI),
       PauliElement(n=4, weight=0.157990+(-0.000000)j, IIIZ),
       PauliElement(n=4, weight=0.173961+(-0.000000)j, IIXZ),
       PauliElement(n=4, weight=-0.259495+(-0.000000)j, IIZI),
       PauliElement(n=4, weight=0.266657+(0.000000)j, IIZZ),
       PauliElement(n=4, weight=-0.156808+(0.000000)j, IZII),
       PauliElement(n=4, weight=-0.173961+(0.000000)j, IZXI),
       PauliElement(n=4, weight=-0.332895+(0.000000)j, IZIZ),
       PauliElement(n=4, weight=-0.173961+(0.000000)j, IZXZ),
       PauliElement(n=4, weight=0.084590+(-0.000000)j, IZZI),
       PauliElement(n=4, weight=0.260677+(-0.000000)j, IZZZ),
       PauliElement(n=4, weight=-0.087688+(-0.000000)j, ZIII),
       PauliElement(n=4, weight=-0.173961+(0.000000)j, ZIXI),
       PauliElement(n=4, weight=0.089715+(0.000000)j, ZIIZ),
       PauliElement(n=4, weight=-0.173961+(0.000000)j, ZIXZ),
       P

In [55]:
pp001.coef_matrix

matrix([[ 0.68414219-1.06136736e-18j,  0.        +0.00000000e+00j,
          0.17396062-9.13155925e-19j,  0.        +0.00000000e+00j,
          0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
          0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
          0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
          0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
          0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
          0.        +0.00000000e+00j,  0.        +0.00000000e+00j],
        [ 0.        +0.00000000e+00j,  0.15798952-4.41326285e-18j,
          0.        +0.00000000e+00j,  0.17396062-9.13155925e-19j,
          0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
          0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
          0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
          0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
          0.        +0.00000000e+00j,  0.        +0.00000000e

## Addition

**Comparsion: Matrix and set methods**

Qubits = 5

- Matrix: 58.3 ms ± 2.4 ms
- Set: 13.2 ms ± 111 µs

Qubits = 8

- Matrix: 3.68 s ± 67 ms per loop
- Set: 2 s ± 74.5 ms

In [30]:
%%timeit
pp001+pp002

18.3 µs ± 279 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [31]:
%%timeit
spa001+spa002

40 µs ± 422 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Mat multiplication

In [12]:
%%timeit
pp001@pp002

NameError: name 'pp001' is not defined

In [36]:
%%timeit
spa001@spa002 # Unable to multiplicate.

138 ms ± 5.14 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Summary

A(xN.N): B took N.N times more slower than A.

|Opeartion|Faster|
|:-------:|:----:|
|o->mat   | Qiskit(x1.2) | 
|mat->o   | PauliPoly(x2-1.2) | 
|Addition | PauliPoly(x2) |
|Mat Mul  | PauliPoly(x Exponential) |
